In [ ]:
from datetime import date, timedelta    # look for posts from each day
import requests                         # send HTTP requests to get HTML from webpages
from bs4 import BeautifulSoup           # search HTML from webpages
from time import sleep                  # give the server a pause between requests
import pandas as pd                     # convert data into dataframe

In [ ]:
# loop through dates to create all initial urls for the last year
dates = []

# loop through urls to create array of all posts per day, for the last year
urls_list = []

In [ ]:
url = "https://statmodeling.stat.columbia.edu/"


# start_date = date(2021, 1, 1)           # set first day of the desired time period 
# end_date = date(2022, 1, 1)             # set last day of the desired time period 

delta = end_date-start_date             # 365 days

for i in range(delta.days+1):           # range(0, 365)
  day = start_date + timedelta(days=i)  # 2021-01-01, 2021-01-02, 2021-01-03
  date_string = day.strftime("%Y/%m/%d")# convert to 2021/01/01, 2021/01/02, 2021/01/03
  dates.append(date_string)

for date in dates:
  initial_url = url + date              # https://statmodeling.stat.columbia.edu/2021/01/01
  url_list.append(initial_url)          # ['https://statmodeling.stat.columbia.edu/2021/01/01, ]

In [ ]:
commenters = {}                         # grab all commenters names and counts for one post
unique_commenters = []                  # create one csv of URL, blog title, blog date, and unique comment count

for url in urls_list:
  # grab all post(s) from one day
  headers = {                           # to avoid the error message from the site
      "user-agent": "Chrome/88.0.4324.182"
      }
  site_get = requests.get(url, headers = headers)
  sleep(0.5)                            # 1-second pause between requests
  site_html= BeautifulSoup(site_get.text, "html.parser")

  # find the comment button(s) for all post(s) made on that day
  all_post_urls= site_html.findAll("span", {"class":"comments-link"}) 
                                        # [<span class="comments-link"><a href="https://statmodeling.stat.columbia.edu/2021/01/05/authors-retract-the-nature-communications-paper-on-female-mentors/#comments"><b>15</b> Replies</a></span>, ]

  for post in all_post_urls:

    # find all post link(s) on that day
    # one url that holds all the comments for one post
    url_we_want = post.find("a").get("href")                          
                                        # https://statmodeling.stat.columbia.edu/2021/01/01/what-we-did-in-2020/#comments
    
    # grab the single post content
    url_we_want_get = requests.get(url_we_want, headers = headers)
    # sleep(1)                            # 1-second pause between requests
    one_post_html= BeautifulSoup(url_we_want_get.text, "html.parser")

    post_title = one_post_html.find("h1", {"class": "entry-title"}).text
                                        
    post_date = one_post_html.find("time", {"class": "entry-date"}).get("datetime")
                                        
    # find all the comment(s) for one post
    all_comments_one_post = one_post_html.findAll("div", {"class": "comment-author vcard"})
                                        # [<div class="comment-author vcard">
                                        # <img alt="" class="avatar avatar-68 photo" height="68" loading="lazy" src="https://secure.gravatar.com/avatar/4ca887fa9f168857d43ef5eb3c636296?s=68&amp;d=mm&amp;r=g" srcset="https://secure.gravatar.com/avatar/4ca887fa9f168857d43ef5eb3c636296?s=136&amp;d=mm&amp;r=g 2x" width="68"/><span class="fn"><a class="url" href="https://twitter.com/JaneSmith" rel="external nofollow ugc">Jane Smith</a></span> on <a href="https://statmodeling.stat.columbia.edu/2021/01/01/what-we-did-in-2020/#comment-1630101"><time datetime="2021-01-01T09:16:18-05:00">January 1, 2021 9:16 AM at 9:16 am</time></a> <span class="says">said:</span>
                                        # </div>, <div class="comment-author vcard">

    # create a python dictionary that tracks unique commenters
    unique_commenters_for_one_post = {}

    for comment in all_comments_one_post:
      # find the commenter's name
      commenter_name = comment.find("span", {"class": "fn"}).text
                                        # looks inside <span class="fn"><a class="url" href="https://twitter.com/JaneSmith" rel="external nofollow ugc">Jane Smith</a></span>
                                        # for Jane Smith

      if commenter_name in unique_commenters_for_one_post:
        unique_commenters_for_one_post[commenter_name] += 1
      else:
        unique_commenters_for_one_post[commenter_name] = 1
                                        # { 'Jane Smith': 1, 'Jane Doe': 2, }

    unique_commenters.append([
                              post_title,
                                        # What we did in 2020, and thanks to all our collaborators and many more
                              post_date,
                                        # 2021-01-01T09:00:09-05:00
                              len(unique_commenters_for_one_post.keys()),
                                        # how many unique commenters there were on this post: 12
                              url_we_want     
                                        # 'https://statmodeling.stat.columbia.edu/2021/01/01/what-we-did-in-2020/#comments'
                              ])

    commenters[url_we_want] = unique_commenters_for_one_post
                                        # { {'Jane Smith': 1, 'Jane Doe': 2, } }

In [ ]:
from google.colab import files

# convert to a pandas dataframe
unique_commenters_df = pd.DataFrame(
  unique_commenters, 
  columns= [
            "Post Title", 
            "Post Date",
            "Unique Commenters", 
            "URL"
            ])

year = 2021                             # set the year to title your csv file
csv_file = str(year) + '_blog.csv'

# convert to csv
with open(csv_file, 'w') as f:
  unique_commenters_df.to_csv(
    f, 
    header = True, 
    index=False
    )

# download the csv
files.download(csv_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>